# Project 3: Predicting 7 Day Retention in Gaming

In [21]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [29]:
import sklearn
print(sklearn.__version__)

1.6.1


In [22]:
ogb_df = pd.read_csv("online_gaming_behavior_dataset.csv")
ogb_df

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40029,49029,32,Male,USA,Strategy,20.619662,0,Easy,4,75,85,14,Medium
40030,49030,44,Female,Other,Simulation,13.539280,0,Hard,19,114,71,27,High
40031,49031,15,Female,USA,RPG,0.240057,1,Easy,10,176,29,1,High
40032,49032,34,Male,USA,Sports,14.017818,1,Medium,3,128,70,10,Medium


In [23]:
ogb_df.isnull().sum()

PlayerID                     0
Age                          0
Gender                       0
Location                     0
GameGenre                    0
PlayTimeHours                0
InGamePurchases              0
GameDifficulty               0
SessionsPerWeek              0
AvgSessionDurationMinutes    0
PlayerLevel                  0
AchievementsUnlocked         0
EngagementLevel              0
dtype: int64

In [24]:
ogb_df.drop(columns=['PlayerID'], inplace=True)
ogb_df

,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium
1,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium
2,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High
3,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium
4,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...
40029,32,Male,USA,Strategy,20.619662,0,Easy,4,75,85,14,Medium
40030,44,Female,Other,Simulation,13.539280,0,Hard,19,114,71,27,High
40031,15,Female,USA,RPG,0.240057,1,Easy,10,176,29,1,High
40032,34,Male,USA,Sports,14.017818,1,Medium,3,128,70,10,Medium


In [25]:
print(ogb_df['GameDifficulty'].unique())
print(ogb_df['EngagementLevel'].unique())

['Medium' 'Easy' 'Hard']
['Medium' 'High' 'Low']


In [26]:
ogb_df = pd.get_dummies(ogb_df, columns=['GameGenre', 'Gender', 'Location'], drop_first=True)
le_difficulty = LabelEncoder()
le_engagement = LabelEncoder()
ogb_df['GameDifficulty'] = le_difficulty.fit_transform(ogb_df['GameDifficulty'])
ogb_df['EngagementLevel'] = le_engagement.fit_transform(ogb_df['EngagementLevel'])

In [27]:
ogb_df['retention_7'] = ogb_df['SessionsPerWeek'] * ogb_df['AvgSessionDurationMinutes']
x = ogb_df.drop(columns='retention_7')
y = ogb_df['retention_7']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse:.4f}")

TypeError: got an unexpected keyword argument 'squared'